In [1]:
# 構造のメタデータを保存するDBを作成する

In [2]:
from sqlalchemy import create_engine, Column, Integer, String, Sequence
from sqlalchemy.orm import declarative_base
import uuid

path2db = "/Users/y1u0d2/desktop/Lab/data/qe_data/SiO2"

Base = declarative_base()

class Structure(Base):
    __tablename__ = 'structure'
    id = Column(String, primary_key=True)
    original_path = Column(String, unique=True)
    structure_id = Column(Integer)
    structure_name = Column(String)
    calculation_type = Column(String)

# データベースエンジンの作成とテーブルの初期化
engine = create_engine(f'sqlite:///{path2db}/structure.db')
Base.metadata.create_all(engine)

# データの追加

In [3]:
from glob import glob
import re

def get_mpid_from_path(path):
    match = re.search(r"mp-\d+", path)
    if match:
        result = match.group(0)
    else:
        result = None
    return result

def generate_unique_uuid(session):
    while True:
        new_uuid = str(uuid.uuid4())
        existing = session.query(Structure).filter_by(id=new_uuid).first()
        if not existing:
            return new_uuid

In [4]:
from sqlalchemy.orm import sessionmaker
import os
import shutil
from mlptools.io.read import read_from_format
import pickle


path2scf = "/Users/y1u0d2/desktop/Lab/result/qe/SiO2/mp-6947/result"
path2atoms = "/Users/y1u0d2/desktop/Lab/data/qe_data/SiO2/atoms"
calculation_type = "scf"
structure_name = "stishovite"

all_scf_dirs = []
all_scf_dirs.extend(glob(f'{path2scf}/scf*'))


Session = sessionmaker(bind=engine)
session = Session()

for scf_dir in all_scf_dirs[:100]:
    mpid = get_mpid_from_path(scf_dir)
    # ディレクトリのバリデーション
    try:
        if os.path.exists(os.path.join(scf_dir, "atoms.pkl")):
            print(f"atoms.pkl already exists {scf_dir}")
            atoms = pickle.load(open(f"{scf_dir}/atoms.pkl", 'rb'))
        else:
            atoms = read_from_format(
                path2target=scf_dir,
                format='espresso-in',
                structure_id=mpid
            )
            with open(f"{scf_dir}/atoms.pkl", 'wb') as f:
                pickle.dump(atoms, f)   
            print(f"Successfully created atoms.pkl {scf_dir}") 
    except:
        print(f"[Error]: Invalid scf directory {scf_dir}")
        # shutil.rmtree(scf_dir)
        # print(f"[Remove dir]: {scf_dir}")
        continue

    # DBへ追加
    unique_id = generate_unique_uuid(session)
    try:
        structure = Structure(
            id=unique_id,
            original_path=scf_dir, 
            structure_id=mpid, 
            structure_name=structure_name,
            calculation_type=calculation_type
        )
        session.add(structure)
        session.commit()

        # create directory and copy files
        path2target = os.path.join(path2atoms, structure.id)
        # copy scf.in, scf.out and atoms.pkl
        os.makedirs(path2target)
        if calculation_type == "scf":
            shutil.copy(os.path.join(scf_dir, "scf.in"), path2target)
            shutil.copy(os.path.join(scf_dir, "scf.out"), path2target)
            shutil.copy(os.path.join(scf_dir, "atoms.pkl"), path2target)
        elif calculation_type == "relax":
            shutil.copy(os.path.join(scf_dir, "atoms.pkl"), path2target)
        print(f"[DB COMMIT SUCCESS]: {path2target}")
    except Exception as e:
        print(f"[Error]: {e}")
        session.rollback()

session.close()

Already exists /Users/y1u0d2/desktop/Lab/result/qe/SiO2/mp-6947/result/scf_2366
[DB COMMIT SUCCESS]: /Users/y1u0d2/desktop/Lab/data/qe_data/SiO2/dev_atoms/2d42c63a-abdf-4d1d-86b0-22ddd013ae28
Already exists /Users/y1u0d2/desktop/Lab/result/qe/SiO2/mp-6947/result/scf_5715
[DB COMMIT SUCCESS]: /Users/y1u0d2/desktop/Lab/data/qe_data/SiO2/dev_atoms/28c1c2ed-6645-4fc5-8dd3-4b33c7305fc6
Already exists /Users/y1u0d2/desktop/Lab/result/qe/SiO2/mp-6947/result/scf_12648
[DB COMMIT SUCCESS]: /Users/y1u0d2/desktop/Lab/data/qe_data/SiO2/dev_atoms/24a89245-a85a-4100-913d-a772af8a2125
Already exists /Users/y1u0d2/desktop/Lab/result/qe/SiO2/mp-6947/result/scf_18372
[DB COMMIT SUCCESS]: /Users/y1u0d2/desktop/Lab/data/qe_data/SiO2/dev_atoms/41b77c7b-0b12-410f-91f7-814e6cf792b4
Already exists /Users/y1u0d2/desktop/Lab/result/qe/SiO2/mp-6947/result/scf_1341
[DB COMMIT SUCCESS]: /Users/y1u0d2/desktop/Lab/data/qe_data/SiO2/dev_atoms/74dcdaed-1c66-467f-a303-f76e5bc1ca48
Already exists /Users/y1u0d2/desktop/L